In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa

In [5]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_meta = pd.read_csv(trailers_csv, index_col=None, header=0)
display(trailers_meta.head())

,Unnamed: 0,Unnamed: 0.1,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,0,0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,1,4,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,2,5,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,3,6,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,4,7,[4k] Покемон. Детектив Пикачу,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [54]:
info_array = []
min_len = 100
max_len = 0
# del_array_id = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(trailers_meta)):
    name = trailers_meta.iloc[i]['trailers_name']
    content_id = trailers_meta.iloc[i]['content_id']
    try:
        info_csv = info_folder + '/' + name + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info_df = info_df.iloc[: , 3:]
        info_array.append(info_df.to_numpy())
#         print(info_array[i])
        if len(info_array[i]) < min_len:
            min_len = len(info_array[i])
            min_name = name
        if len(info_array[i]) > max_len:
            max_len = len(info_array[i])
            max_name = name
    except:
        del_array_id.append(content_id)
        
print(min_len, max_len)
padded = []
for i in range (len(info_array)):
    padded.append(pad(info_array[i], max_len, 23))
print(padded)

3 396


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
def pad(array, max_len, max_h):
    new_arr = []
    for i in range(max_len):
        new_arr_j = []
        if (i < array.shape[0]):
            for j in range(max_h):
                if j < array.shape[1]:
                    new_arr_j.append(array[i][j])
                else:
                    new_arr_j.append(0)
        else:
            for j in range(max_h):
                 new_arr_j.append(0)
        new_arr.append(new_arr_j)
    return new_arr

In [61]:
shape = np.array(padded).shape

In [68]:
#encoder
encoder_input = Input(shape=(None, shape[0]))
encoder_LSTM1 = LSTM(shape[0], return_sequences=True, dropout=0.2)
encoder_outputs = encoder_LSTM1(encoder_input)
encoder_LSTM2 = LSTM(249, return_state=True)
encoder_outputs, encoder_h, encoder_c = encoder_LSTM2(encoder_outputs)
encoder_states = [encoder_h, encoder_c]

In [69]:
#decoder
decoder_input = Input(shape=(None, shape[0]))
decoder_LSTM1 = LSTM(249, return_sequences=True, dropout=0.2)
decoder_output = decoder_LSTM1(decoder_input, initial_state=encoder_states)
decoder_LSTM2 = LSTM(128, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_LSTM2(decoder_output)
decoder_dense = Dense(128, activation='softmax')
decoder_output = decoder_dense(decoder_output)